In [1]:
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import sys
sys.path.append("..")
sys.path.append("../src")
import os
from src.DS_Generator import DS_Generator
from src.model_evaluation import mean_average_precision
from data_augmentation import *
import numpy as np
import pandas as pd
import tensorflow_datasets as tfds
from util import *
import datetime
%load_ext tensorboard

In [ ]:
train_ds, val_ds = DS_Generator().generate_species_data(TRAIN_SPECIES_DF,seed=5,augment=1,batch_size=32)

In [ ]:
Input = tf.keras.Input((224,224,3))
base = tf.keras.applications.resnet50.ResNet50(weights=None, include_top=False,input_tensor=Input)
flatten = base.output
flatten = tf.keras.layers.Flatten()(flatten)
head = tf.keras.layers.Dense(256, activation="relu")(flatten)
head = tf.keras.layers.Dense(128, activation="relu")(head)
head = tf.keras.layers.Dense(64, activation="relu")(head)
head = tf.keras.layers.Dense(30,activation="softmax")(head)
model = tf.keras.Model(inputs=Input, outputs=head,name="Resnet50ohneimagenet")
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),loss="categorical_crossentropy",metrics=["acc"])

In [ ]:
if model.name not in os.listdir("../../models/"):
    os.makedirs("../../models/"+model.name)
    os.makedirs("../../models/"+model.name+"/logs")
    os.makedirs("../../models/"+model.name+"/saves")
    
time_stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

log_dir = "../../models/"+model.name+"/logs/" +time_stamp 

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

checkpoint_path = "../../models/"+model.name+"/saves/" +time_stamp +"/cp-{epoch:04d}.ckpt"

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True)

In [ ]:
model.fit(
train_ds,
epochs=15,
validation_data=val_ds,
callbacks=[cp_callback,tensorboard_callback])